In [1]:
import numpy as np
from functools import reduce

# Déclaration de variables et de fonctions basiques

rcon = ["02","04","08","10","20","40","80","1B","36"]
sbox = "63 7c 77 7b f2 6b 6f c5 30 01 67 2b fe d7 ab 76\
 ca 82 c9 7d fa 59 47 f0 ad d4 a2 af 9c a4 72 c0\
 b7 fd 93 26 36 3f f7 cc 34 a5 e5 f1 71 d8 31 15\
 04 c7 23 c3 18 96 05 9a 07 12 80 e2 eb 27 b2 75\
 09 83 2c 1a 1b 6e 5a a0 52 3b d6 b3 29 e3 2f 84\
 53 d1 00 ed 20 fc b1 5b 6a cb be 39 4a 4c 58 cf\
 d0 ef aa fb 43 4d 33 85 45 f9 02 7f 50 3c 9f a8\
 51 a3 40 8f 92 9d 38 f5 bc b6 da 21 10 ff f3 d2\
 cd 0c 13 ec 5f 97 44 17 c4 a7 7e 3d 64 5d 19 73\
 60 81 4f dc 22 2a 90 88 46 ee b8 14 de 5e 0b db\
 e0 32 3a 0a 49 06 24 5c c2 d3 ac 62 91 95 e4 79\
 e7 c8 37 6d 8d d5 4e a9 6c 56 f4 ea 65 7a ae 08\
 ba 78 25 2e 1c a6 b4 c6 e8 dd 74 1f 4b bd 8b 8a\
 70 3e b5 66 48 03 f6 0e 61 35 57 b9 86 c1 1d 9e\
 e1 f8 98 11 69 d9 8e 94 9b 1e 87 e9 ce 55 28 df\
 8c a1 89 0d bf e6 42 68 41 99 2d 0f b0 54 bb 16".split(" ")

  # Transformation de la chaîne en liste de hex
def toHexList(String):
  l = []
  for character in String:
      l.append(character.encode('utf-8').hex())
  return l

  # Création d'une liste avec des sous listes de longueur 4
def toSubList(List):
  lf = []
  for i in range(1,int(len(List)/4)+1):
      lf.append(List[i*4-4:i*4])
  return lf

#Décalage à gauche de x pour la dernière ligne
def left_shift(list,offset):
  return np.roll(list[-4:],offset)


In [2]:
# "Thats my Kung Fu" Translation into Hex / Affichage de w[3]
key0 = toHexList("Thats my Kung Fu")
print(f"w[3] : {key0[-4:]}")

w[3] : ['67', '20', '46', '75']


In [3]:
#Circular byte left shift of w[3]
left_shift = left_shift(key0,-1)
print(f"circular byte left shift of w[3]: {left_shift}")

circular byte left shift of w[3]: ['20' '46' '75' '67']


In [4]:
# Byte Substitution (S-Box):
sbox_16SubList = [sbox[i:i+16] for i in range(0, len(sbox), 16)]
SubLine = []
for h in left_shift:
    SubLine.append(sbox_16SubList[int(h[0],16)][int(h[1],16)])
print(f"Byte Substitution (S-Box): {SubLine}")

Byte Substitution (S-Box): ['b7', '5a', '9d', '85']


In [5]:
#Adding round Constant
Add_round = [hex(int(SubLine[0],16)^int("01",16))[2:]] + SubLine[1:]
print(f"Adding round constant (01, 00, 00, 00) gives: g(w[3]) : {Add_round}")

Adding round constant (01, 00, 00, 00) gives: g(w[3]) : ['b6', '5a', '9d', '85']


In [6]:
# Génération des clés
def genKeys(key0SubList,Add_round):
  keys = []
  for j in range(4):
      if j == 0:
          for i in range(4):
              keys.append(hex(int(key0SubList[0][i],16)^int(Add_round[i],16))[2:])
      else:
          for i in range(4):
              keys.append(hex(int(keys[-4:][0],16)^int(key0SubList[j][i],16))[2:])

  for k in range(9):
      #Création d'une liste avec des sous listes de longueur 4
      lf = []
      #Ajout de zero si manquant
      for i in range(len(keys)):
          if len(keys[i])==1: keys[i] = "0" + keys[i]
          
      lf = toSubList(keys)[-4:]
      #Décalage à gauche de 1 pour la dernière ligne
      left_shift = np.roll(keys[-4:],-1)
      # Byte Substitution
      SubLine = []
      for h in left_shift:
          SubLine.append(sbox_16SubList[int(h[0],16)][int(h[1],16)])
      #Adding round Constant
      Add_round = [hex(int(SubLine[0],16)^int(rcon[k],16))[2:]] + SubLine[1:]
      for j in range(4):
          if j == 0:
              for i in range(4):
                  keys.append(hex(int(lf[0][i],16)^int(Add_round[i],16))[2:])
          else:
              for i in range(4):
                  keys.append(hex(int(keys[-4:][0],16)^int(lf[j][i],16))[2:])
  return keys
key0SubList = toSubList(key0)
keys = genKeys(key0SubList,Add_round)

keysSubList = [key0]
for i in range(10):
    keysSubList.append(keys[16*i:(16*i)+16])
for i in range(11):
  print(keysSubList[i])

['54', '68', '61', '74', '73', '20', '6d', '79', '20', '4b', '75', '6e', '67', '20', '46', '75']
['e2', '32', 'fc', 'f1', '91', '12', '91', '88', 'b1', '59', 'e4', 'e6', 'd6', '79', 'a2', '93']
['56', '08', '20', '07', 'c7', '1a', 'b1', '8f', '76', '43', '55', '69', 'a0', '3a', 'f7', 'fa']
['d2', '60', '0d', 'e7', '15', '7a', 'bc', '68', '63', '39', 'e9', '01', 'c3', '03', '1e', 'fb']
['a1', '12', '02', 'c9', 'b4', '68', 'be', 'a1', 'd7', '51', '57', 'a0', '14', '52', '49', '5b']
['b1', '29', '3b', '33', '05', '41', '85', '92', 'd2', '10', 'd2', '32', 'c6', '42', '9b', '69']
['bd', '3d', 'c2', '87', 'b8', '7c', '47', '15', '6a', '6c', '95', '27', 'ac', '2e', '0e', '4e']
['cc', '96', 'ed', '16', '74', 'ea', 'aa', '03', '1e', '86', '3f', '24', 'b2', 'a8', '31', '6a']
['8e', '51', 'ef', '21', 'fa', 'bb', '45', '22', 'e4', '3d', '7a', '06', '56', '95', '4b', '6c']
['bf', 'e2', 'bf', '90', '45', '59', 'fa', 'b2', 'a1', '64', '80', 'b4', 'f7', 'f1', 'cb', 'd8']
['28', 'fd', 'de', 'f8', '6d',

In [7]:
# Fonctions pour les tours de chiffrement

# Mix Columns Standard en entré une ligne de la matrice
def mixColumns(a, b, c, d):
    l =[]
    l.append('{:02x}'.format(gmul(a, 2) ^ gmul(b, 3) ^ gmul(c, 1) ^ gmul(d, 1)))
    l.append('{:02x}'.format(gmul(a, 1) ^ gmul(b, 2) ^ gmul(c, 3) ^ gmul(d, 1)))
    l.append('{:02x}'.format(gmul(a, 1) ^ gmul(b, 1) ^ gmul(c, 2) ^ gmul(d, 3)))
    l.append('{:02x}'.format(gmul(a, 3) ^ gmul(b, 1) ^ gmul(c, 1) ^ gmul(d, 2)))
    return l

def gmul(a, b):
    if b == 1:
        return a
    tmp = (a << 1) & 0xff
    if b == 2:
        return tmp if a < 128 else tmp ^ 0x1b
    if b == 3:
        return gmul(a, 2) ^ a

#Add_Round_Key en entré la matrice et la clé actuelle
def Add_Round_key(stateMatrix,RoundkeyMatrix):
  newStateMatrix = []
  for i in range(len(stateMatrix)):
    newStateMatrix.append(hex(int(stateMatrix[i],16)^int(RoundkeyMatrix[i],16))[2:])
    if len(newStateMatrix[i]) == 1: newStateMatrix[i] = "0" + newStateMatrix[i]
  return(newStateMatrix)

#Sub_Bytes
def Sub_Bytes(StateMatrix):
  newStateMatrix = []
  for h in StateMatrix:
    newStateMatrix.append(sbox_16SubList[int(h[0],16)][int(h[1],16)])
  return newStateMatrix

#Shift_Row
def shift_Row(StateMatrix):
    middleStateMatrix = []
    newStateMatrix = []
    for i in range(4):
      for j in range(4):
        middleStateMatrix.append(StateMatrix[(j*4)+i])
    for i in range(4):
      newStateMatrix.append(np.roll(toSubList(middleStateMatrix)[i],-i))
    return newStateMatrix

# Mix Columon spécifique
def mix_Column(StateMatrix):
  middleStateMatrix = []
  newStateMatrix = []
  for i in range(4):
    for j in range(4):
      middleStateMatrix.append(StateMatrix[j][i])
  middleStateMatrix = toSubList(middleStateMatrix)
  for i in range(4):
    newStateMatrix.append(mixColumns(int(middleStateMatrix[i][0],16),int(middleStateMatrix[i][1],16),int(middleStateMatrix[i][2],16),int(middleStateMatrix[i][3],16)))
  flat_newStateMatrix = [item for sublist in newStateMatrix for item in sublist]
  return flat_newStateMatrix

#Fonction pour les 10 tours de chiffrement en entré le premier texte et la liste de clés
def cipher_round(plainList,keySubList):
  middle_ciphertext = []
  for i in range(11):
    addround = Add_Round_key(plainList,keySubList[i])
    subbytes = Sub_Bytes(addround)
    shiftrow= shift_Row(subbytes)
    if i == 9:
      for k in range(4):
        for j in range(4):
          middle_ciphertext.append(shiftrow[j][k])
      ciphertext = Add_Round_key(middle_ciphertext,keySubList[i+1])
      return ciphertext
    plain = mix_Column(shiftrow)
    plainList = []
    plainList = plain

In [8]:
plainList=toHexList("Two One Nine Two")
print(cipher_round(plainList,keysSubList))

['29', 'c3', '50', '5f', '57', '14', '20', 'f6', '40', '22', '99', 'b3', '1a', '02', 'd7', '3a']
